# Week 2

In [1]:

# Rijen filteren
# gdf_weg = gdf_weg[gdf_weg.function.isin(['transitie', 'voetpad', 'parkeervlak', 'voetgangersgebied', 'woonerf'])]
# gdf_weg = gdf_weg[gdf_weg.functieWegdeel == 'waardeOnbekend']


# # Kolom toevoegen voor afstand van een boom tot de rand van een polygon
# tmp = to_crs(bomen.copy(), cea)
# tmp['punten'] = tmp.geometry.astype(str).copy()
# tmp = sjoin(to_crs(gdf.copy(), cea), tmp, how='inner')
# tmp = tmp.geometry.exterior.distance(GeoDataFrame(tmp.punten, geometry=tmp.punten.apply(wkt.loads), crs=cea))
# # gdf['afstand_tot_rand'] = tmp.values
# # del tmp

# # Kolom toevoegen voor aantal bomen per polygon
# gdf = gdf.merge(sjoin(gdf, bomen, how='left')
#                 .groupby(['gml_id'], as_index=False).sum()[[
#                     'gml_id', 'aantal_bomen', 'boomoppervlakte', 'afstand_tot_rand']], 
#                 how='left', on='gml_id')
# gdf['aantal_bomen'] = gdf.aantal_bomen.astype(int)
# gdf['afstand_tot_rand'] = gdf.afstand_tot_rand / gdf.aantal_bomen



In [ ]:


# bomen_per_functieWegdeel = gdf[['functieWegdeel', 'aantal_bomen']].groupby(['functieWegdeel'], as_index=False).sum()


# # Sommige geometrieën zijn LineStrings, hiervan maken we Polygons zodat we ook de oppervlakte kunnen berekenen
# gdf.loc[gdf.oppervlakte == 0, 'geometry'] = gdf.loc[gdf.oppervlakte == 0, 'geometry'].apply(
#     lambda x: 'POLYGON ((' + ', '.join(x.wkt[12:-1].split(', ') + x.wkt[12:-1].split(', ')[::-1]) + '))').apply(
#     wkt.loads)
# gdf.loc[gdf.oppervlakte == 0, 'oppervlakte'] = calc_area(gdf[gdf.oppervlakte == 0], m=True).oppervlakte


# gdf = sjoin(gdf, buurten[['geometry', 'Buurtcode', 'Buurt', 'Buurtcombinatie_code', 'Stadsdeel_code']], 
#             how='left').drop(columns=['index_right'])
# gdf = gdf[~gdf.Buurtcode.isna()]  # elke weg die buiten de buurten valt laten we weg (we gebruiken hier geen overlay)

# # Beschikbare ruimte, gedefinieerd als het verschil tussen oppervlakte van een wegdeel en oppervlakte van bestaande bomen
# tmp['ruimte'] = (tmp.oppervlakte - tmp.boomoppervlakte)
# tmp.loc[tmp.ruimte < 0, 'ruimte'] = 0  # Beschikbare ruimte kan niet negatief zijn

# categorical_features = ['aantal_bomen', 'Stadsdeel_code']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1337)
# categorical_transformer = Pipeline([('onehot', OneHotEncoder(handle_unknown='ignore'))])
# preprocessor = ColumnTransformer([('cat', categorical_transformer, categorical_features)])
# knn = Pipeline([('preprocessor', preprocessor), ('knn', KNeighborsClassifier())])

# result_knn['diff'] = result_knn.where(result_knn.boom == result_knn.predicted)
# print(f'{len(result_knn)} - {result_knn["diff"].sum()} = {len(result_knn) - result_knn["diff"].sum()}')




In [ ]:

# ML
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Machine learning

# K-nearest neighbor wordt gebruikt om de aanwezigheid van bomen per wegdeelfunctie te voorspellen. 


# Get the data
tmp = gdf[['function', 'aantal_bomen']].copy()
tmp['aantal_bomen'] = tmp.aantal_bomen.where(tmp.aantal_bomen == 0, 1).astype(int)
X = tmp[['aantal_bomen']].copy()  # categorisch: wel/niet bomen

# Create the outcome variable
y = LabelEncoder().fit_transform(tmp.function)
encoded_labels = pd.DataFrame(dict(y=y, func=tmp.function)).groupby('func').mean()

# Create the model
knn = KNeighborsClassifier()
knn.fit(X, y)

# Predict 
tmp = tmp[tmp.aantal_bomen == 0]
functie_proba = knn.predict_proba(tmp[['aantal_bomen']])
encoded_labels['proba'] = encoded_labels.y.apply(lambda x: functie_proba[:, x].mean())
encoded_labels


# Ridge-regressie wordt gebruikt om het aantal bomen te voorspellen aan de hand van wegdeeloppervlakte


tmp = gdf[['oppervlakte', 'function', 'Stadsdeel_code', 'aantal_bomen']].copy()
tmp = tmp[tmp.aantal_bomen > 0]

categorical_features = ['function', 'Stadsdeel_code']
numeric_features = ['oppervlakte']

X = tmp[categorical_features + numeric_features]
y = tmp.aantal_bomen
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Create the model
numeric_transformer = Pipeline([('scaler', StandardScaler())])
categorical_transformer = Pipeline([('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)])
reg = Pipeline([('preprocessor', preprocessor), ('ridge', Ridge())])
reg.fit(X_train, y_train)

# Predict
result_reg = pd.DataFrame(y_test, columns=['aantal_bomen'])
result_reg['predicted'] = reg.predict(X_test).astype(int)
result_reg['diff'] = result_reg.aantal_bomen - result_reg.predicted
print(f'RMSE = {np.sqrt(((result_reg["diff"])**2).values.mean())}')
result_reg.sort_values(['diff'], ascending=False)


In [ ]:
# # def explode(indata):
    
# #     indf = gpd.GeoDataFrame.from_file(indata)
# #     outdf = gpd.GeoDataFrame(columns=indf.columns)
    
# #     for idx, row in indf.iterrows():
        
# #         if type(row.geometry) == Polygon:
# #             outdf = outdf.append(row,ignore_index=True)
        
# #         if type(row.geometry) == MultiPolygon:
# #             multdf = gpd.GeoDataFrame(columns=indf.columns)
# #             recs = len(row.geometry)
# #             multdf = multdf.append([row]*recs,ignore_index=True)
            
# #             for geom in range(recs):
# #                 multdf.loc[geom,'geometry'] = row.geometry[geom]
# #             outdf = outdf.append(multdf,ignore_index=True)
    
# #     return outdf

# def explode(gdf):
#     """ 
#     Explodes a geodataframe 
    
#     Will explode muti-part geometries into single geometries. Original index is
#     stored in column level_0 and zero-based count of geometries per multi-
#     geometry is stored in level_1
    
#     Args:
#         gdf (gpd.GeoDataFrame) : input geodataframe with multi-geometries
        
#     Returns:
#         gdf (gpd.GeoDataFrame) : exploded geodataframe with a new index 
#                                  and two new columns: level_0 and level_1
        
#     """
#     gs = gdf.explode()
#     gdf2 = gs.reset_index().rename(columns={0: 'geometry'})
#     gdf_out = gdf2.merge(gdf.drop('geometry', axis=1), left_on='level_0', right_index=True)
#     gdf_out = gdf_out.set_index(['level_0', 'level_1']).set_geometry('geometry')
#     gdf_out.crs = gdf.crs
#     return gdf_out



In [ ]:
# Vanaf hier is het louter geklooi.

# Ik probeerde te onderzoeken hoe ik een plot kan maken waarbij de nieuwe bomen (met 257 tegelijk, in 34 iteraties) uit de grond zouden schieten 


print(f'{int(len(geo)/34)} × {int(len(geo)/257)} = {len(geo)}')


fig = plt.figure()
plt.scatter(geo.apply(lambda p: p.x), geo.apply(lambda p: p.y))
plt.axis('off');


def main():
    x = geo.apply(lambda p: p.x)
    y = geo.apply(lambda p: p.y)
    data = np.array((x, y))
    
    fig = plt.figure()
    scat = plt.scatter(x, y)

    ani = animation.FuncAnimation(fig, update_plot, frames=257, fargs=(data, scat))
    plt.show()

def update_plot(i, data, scat):
    scat.set_array(data[i-257:i])
    return scat,

main()


%matplotlib notebook

xdata = []
ydata = []

scat = plt.scatter(xdata, ydata)

def update(i):
    xdata.append(geo.apply(lambda p: p.x)[i-257:i])
    ydata.append(geo.apply(lambda p: p.y)[i-257:i])
    scat.set_array(np.array((xdata, ydata)))
    return scat

ani= matplotlib.animation.FuncAnimation(plt.gcf(), update, frames=257, interval=257, repeat=False)

plt.show()


%matplotlib notebook 

fig = plt.figure()
plt.ion()  # enable interactivity
plt.axis('off')

x = list()
y = list()

for i in range(257, len(geo), 257):
    x.append(geo.apply(lambda p: p.x)[i-257:i])
    y.append(geo.apply(lambda p: p.y)[i-257:i])
    plt.scatter(x, y)
    plt.pause(.1)


fig, ax = plt.subplots(1, figsize=(50, 50))
plt.axis('off')
gdf.plot(ax=ax, color='#1f77b4')
# bomen.plot(ax=ax, color='#bcbd22', markersize=50)
# geo = GeoSeries(gdf.points, crs=cea)
# geo.plot(ax=ax, color='#d62728', markersize=50);
# del geo


fig, ax = plt.subplots(1, figsize=(50, 50))
plt.axis('off')
gdf.plot(ax=ax)
plt.savefig('Amsterdamse_wegen.png')


# Lengte en breedte

# In deze code probeer ik de afmetingen van een polygon te benaderen (met de nadruk op benaderen)


tmp = to_crs(gdf, cea)

item = 4
minx, miny, maxx, maxy = tmp.geometry[item].bounds

lengte1 = sqrt((maxx - minx)**2 + (maxy - miny)**2)
opp = tmp.geometry[item].area
breedte1 = opp / lengte1
print(f'{lengte1} x {breedte1} = {opp}')

lengte2 = maxx - minx
breedte2 = maxy - miny
ratio = opp / (lengte2*breedte2)
print(f'{lengte2*sqrt(ratio)} x {breedte2*sqrt(ratio)} = {opp}')

print(f'{mean((lengte1, lengte2))} x {mean((breedte1, breedte2))} = {opp}')

tmp.geometry[item]

del tmp

# Week 1

In [1]:
# # Importeer BAG-data
# real_estate_data = pd.read_csv('data/real_estate_data.csv', dtype=str)
# bag = pd.read_csv('data/BAG_data_Amsterdam.csv', dtype=str)

# # Importeer groene daken
# df = pd.read_csv('data/daken/GROENE_DAKEN.csv', sep=';')
# df['geometry'] = df.WKT_LNG_LAT.str.replace(',', ' ')
# df = GeoDataFrame(df, crs=crs, geometry=df.geometry.apply(wkt.loads)).\
#     drop(columns=['OBJECTNUMMER', 'WKT_LNG_LAT', 'WKT_LAT_LNG', 'Unnamed: 14'])

In [2]:
# # Importeer gemeentelijke data

# grond = pd.read_csv('data/GRONDGEBRUIK_2017.csv', sep=';', usecols=['AMScode_Oms', 'WKT_LNG_LAT'])
# grond.rename(columns={'WKT_LNG_LAT':'geometry'}, inplace=True)
# grond = GeoDataFrame(grond, crs=crs, geometry=grond.geometry.apply(wkt.loads))

# # bomen = pd.concat([pd.read_csv(file, sep=';', usecols=['WKT_LNG_LAT'], dtype=str) 
# #                    for file in glob.glob('data\BOMEN*.csv')], axis=0, ignore_index=True)
# # bomen['geometry'] = bomen.WKT_LNG_LAT.str.replace(',', ' ')
# # bomen = GeoDataFrame(bomen, crs=crs, geometry=bomen.geometry.apply(wkt.loads)).drop(columns='WKT_LNG_LAT')

# # merged = sjoin(grond, bomen).groupby('AMScode_Oms').count()

In [3]:
# # Maak een dataframe met gematchte postcodes en stadsdelen, en bijbehorende polygonen
# polygons = pc6.merge(buurten[['Buurtcode', 'Stadsdeel_code']], on=['Buurtcode']).\
#     merge(sd[['Stadsdeel', 'Stadsdeel_code', 'geometry']], on=['Stadsdeel_code'], suffixes=('_PC', '_SD')).\
#     drop(columns=[
# #     'Buurtcode', 
#     'Buurtcombinatie_code', 
#     'Stadsdeel_code'])
# polygons.to_csv('polygons.csv', index=False, encoding='utf8')

In [ ]:
# from math import pi, cos, radians

# def reproject_area(S):
#     earth_radius = 6364.763 
#     C = (pi * earth_radius / 180.0)**2
#     return S*C

# # Berekening volgens Jurjen
# sd = to_crs(sd, crs)
# sd['area1'] = sd.geometry.area
# sd['area1'] = sd.area1.apply(lambda x : reproject_area(x))

# # Berekening volgens Lambert
# sd = to_crs(sd, lambert)
# sd['area3'] = sd.geometry.area / 10**6



## Grondgebruik

In [ ]:
# # Grondgebruik toevoegen
# grondgebruik = sjoin(gdf_pv, grond).drop(columns=['index_right'])
# grondgebruik = grondgebruik.groupby(grondgebruik.index).agg(list)
# codes = grondgebruik.AMScode_Oms.copy()
# grondgebruik = grondgebruik.drop(columns=['AMScode_Oms']).applymap(lambda x: x[0])
# grondgebruik['AMScode_Oms'] = codes.copy()
# # # We gaan ipv grondgebruik, strategische plannen gebruiken
# # to_keep = ['Wegverkeersterrein', 'Wegverkeersterrein rijkswegen', 'Wegverkeersterrein provinciale wegen', 'Woonterrein', 'Detailhandel, winkelcentra en horeca', 'Overheid (kantoor/dienstverlening/openbare voorzieningen)', 'Overheid (nutsbedrijven/defensieterrein)', 'Sociaal-culturele voorzieningen (incl. onderwijs, gezondheidszorg)', 'Bedrijventerrein (industrie, tentoonstellingsterrein, garagebedrijven)', 'Bedrijventerrein (overwegend kantoren/kantoorachtig)', 'Bouwterrein woonbestemming', 'Bouwterrein niet-woonbestemming', 'Semi verhard overig terrein', 'Parkeerterrein ArenA', 'Dagrecreatief terrein', 'Verblijfsrecreatief terrein', 'Open droog-natuurlijk terrein', 'Open nat-natuurlijk terrein', 'Open nat-natuurlijk terrein met schotse hooglanders']
# # to_drop = ['Spoorterrein', 'Metro, vrije trambanen', 'Bedrijfsspoor', 'Vliegveld (niet het gras)', 'Stortplaats', 'Wrakkenopslagplaats', 'Begraafplaats', 'Tijdelijke opslag van grond', 'Park en plantsoen', 'Sportterrein', 'Golfterrein', 'Open nat-natuurlijk terrein waterpartijen', 'IJsselmeer/Markermeer', 'Recreatief binnenwater', 'Recreatief binnenwater grote plas', 'Water in park/plantsoen', 'Water in golfterrein', 'Overig water breder dan 6 m.', 'Overig water `t IJ', 'Overig water grote plassen''Artis', 'Terrein voor glastuinbouw', 'Overig agrarisch terrein', 'Bos', 'Volkstuin']
# # gdf_pv = gdf_pv[~gdf_pv.grondgebruik.isin(to_drop)]

# # gdf_pv = GeoDataFrame(grondgebruik)
# gdf_pv = GeoDataFrame(grondgebruik, crs=crs)
# gdf_pv['geometry'] = gdf_pv.geometry.to_crs(crs)
# gdf_pv.crs = crs

# del grondgebruik, codes

# Plot

In [ ]:
# buurten = to_crs(buurten, crs=mercator)
# centrum_buurten = buurten[buurten.Stadsdeel_code == 'A']

In [ ]:
# # Maak een plot van Amsterdam-Centrum, met groene kleuring naar aantal parkeervakken, en in rood de vakken zelf
# fig, ax = plt.subplots(1, figsize=(60, 60))
# plt.axis('off')
# plt.title('Centrum, aantal parkeervakken per buurt', size=80)
# centrum.plot(ax=ax, column='aantal_vakken', cmap='Greens', linewidth=8, edgecolor='white', scheme='Fisher_Jenks')
# vakken.plot(ax=ax, color='red');

# # Gooi er een paar labels overheen
# for idx, row in centrum.iterrows():
#     plt.annotate(s=row['Buurt'], xy=row['coords'], horizontalalignment='center', size=40);

# fig, ax = plt.subplots(1, figsize=(60, 60))
# plt.axis('off')
# plt.title('Amsterdam, aantal parkeervakken per buurt', size=80)
# sd.plot(ax=ax, facecolor='none', edgecolor='black')
# vakken_per_buurt.plot(ax=ax, column='aantal_vakken', cmap='Greens', linewidth=8, edgecolor='black', scheme='Fisher_Jenks')
# alle_vakken.plot(ax=ax, edgecolor='black');

# Maak een plot van Centrum over de kaart van Amsterdam, met groene kleuring naar aantal parkeervakken
# ax = centrum.plot(column='aantal_vakken', cmap='Accent', edgecolor='black')
# plt.axis('off');
# mplleaflet.display(fig=ax.figure, tiles='cartodb_positron')

In [ ]:
# # Maak een plot van Amsterdam-Centrum met groene kleuring naar aantal plantbare bomen en in rood de vakken zelf

# variable = 'aantal_bomen'

# vmin = centrum[variable].min()
# vmax = centrum[variable].max()

# fig, ax = plt.subplots(1, figsize=(20, 10))

# plt.axis('off')
# plt.title('Centrum, ruimte voor bomen, per buurt', size=20)

# centrum.plot(ax=ax, column=variable, cmap='viridis', linewidth=1, edgecolor='white')
# vakken.plot(ax=ax, linewidth=1, color='red')

# sm = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=vmin, vmax=vmax))
# sm._A = []
# cbar = fig.colorbar(sm)
# cbar.ax.set_ylabel(variable)

# # Gooi er een paar labels overheen
# for idx, row in centrum.iterrows():
#     plt.annotate(s=row['Buurt'], xy=row['coords'], horizontalalignment='center', size=8,
#                 bbox=dict(facecolor='white', edgecolor='white', boxstyle='round,pad=0.1'))